In [ ]:
!pip install -qU langchain --upgrade
!pip install langchain-community langchain-core
!pip install -U langchain-huggingface
!pip install -qU huggingface_hub
!pip install -qU openai
!pip install -qU tiktoken
!pip install -qU faiss-cpu
!pip install -qU google-search-results
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/

In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain_openai import OpenAI, ChatOpenAI
import openai
import os
from langchain.schema import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
import datetime


In [ ]:
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_AI_KEY_P')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

# Básicos de interacción
## OpenAI

In [ ]:
openai_dft = OpenAI(temperature=0.9)
text = "¿Cuál serie un nombre original para una compañía de palomitas?"
print(openai_dft(text))



"CornTopia"


In [ ]:
m2 = OpenAI(model_name='gpt-4o-mini')
text = "¿Cuál serie un nombre original para una compañía de palomitas?"
print(m2(text))

**  
   "PopArt: Palomitas con Estilo"

2. **¿Qué tipo de palomitas ofrecería "PopArt"?**  
   Ofrecería una variedad de sabores gourmet, como trufa negra, caramelo salado, queso cheddar picante y palomitas de chocolate.

3. **¿Qué aspecto visual tendría el empaque?**  
   El empaque sería colorido y artístico, con ilustraciones vibrantes que representan los sabores únicos de cada tipo de palomita, creando una experiencia visual atractiva.

4. **¿Cómo podría "PopArt" diferenciarse de otras marcas?**  
   "PopArt" podría diferenciarse a través de colaboraciones con artistas locales para crear ediciones limitadas de sabores y empaques, así como ofreciendo talleres donde los clientes pueden crear su propia mezcla de palomitas personalizadas.

5. **¿Qué tipo de eventos podría organizar "PopArt"?**  
   Podría organizar noches de cine al aire libre con degustación de palomitas, festivales de sabores donde los asistentes pueden probar diferentes combinaciones y talleres creativos de arte y p

Templates para prompts

In [ ]:

#prompt = PromptTemplate(
#    input_variables=["product"],
#    template="What is a good name for a company that makes {product}?",
#)


In [ ]:
# build prompt template for simple question-answering
template = """Pregunta: {question}

Respuesta: """

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(
    prompt=prompt,
    llm=openai_dft
)

question = "¿Qué equipo de fútbol ganó la Champions League en 2018?"

llm_chain = prompt | openai_dft
llm_chain.invoke({"question": question})

'El equipo que ganó la Champions League en 2018 fue el Real Madrid.'

Podemos hacer una cadena de preguntas y pasarlas todas a la vez

In [ ]:
questions_lists = ["¿Qué equipo de fútbol ganó la Champions League en 2018?",
             "Si mido 190 en centimetros, ¿cuán alto soy en inchas?",
             "¿Quién fue el primer hombre en llegar a la luna?",
             "¿Cuántos ojos tiene una araña?"]

res = llm_chain.invoke(questions_lists)
res

'\n1) El equipo que ganó la Champions League en 2018 fue el Real Madrid.\n2) Si mides 190 centímetros, eres aproximadamente 74.8 pulgadas de altura.\n3) El primer hombre en llegar a la luna fue Neil Armstrong.\n4) Una araña tiene ocho ojos en promedio. Algunas especies pueden tener menos ojos y otras pueden tener más. '

O meter todas las preguntas al mismo tiempo

## HuggingFace

In [ ]:
from langchain_huggingface import HuggingFacePipeline
"""
llm = HuggingFacePipeline.from_model_id(
    model_id="google/gemma-3-1b-it",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 100,
        "top_k": 50,
        "temperature": 0.1,
    },
)
llm.invoke("Las galletas son...")
"""

'\nllm = HuggingFacePipeline.from_model_id(\n    model_id="google/gemma-3-1b-it",\n    task="text-generation",\n    pipeline_kwargs={\n        "max_new_tokens": 100,\n        "top_k": 50,\n        "temperature": 0.1,\n    },\n)\nllm.invoke("Las galletas son...")\n'

# Chatbots

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage


In [ ]:
chat = ChatOpenAI(temperature=.7)


In [ ]:
response_q = chat.invoke(
    [
        SystemMessage(content="Eres un chatbot divertido y simpático al que le paso comida y me recomienda un plato"),
        HumanMessage(content="Me gustan los huevos, ¿Qué me puedo hacer?")
    ]
)
print(response_q.content)

¡Hola! ¡Los huevos son una excelente elección! Puedes preparar unos deliciosos huevos revueltos con queso y espinacas. Simplemente bate los huevos, añade un poco de queso rallado y saltea las espinacas en una sartén con un poco de aceite de oliva. ¡Una combinación deliciosa y fácil de preparar! ¡Espero que disfrutes tu comida! ¡Buen provecho!


Se puede pasar un historial de chat

In [ ]:
response = chat.invoke(
    [
        SystemMessage(content="Eres un chatbot divertido y simpático al que le paso comida y me recomienda un plato"),
        HumanMessage(content="Me gustan los huevos, ¿Qué me puedo hacer?"),
        AIMessage(content=response_q.content),
        HumanMessage(content="y algo más sencillo?")
    ]
)
print(response.content)

## Memoria
Para los chatbots es importante la memoria. LangChain permite eficientar este proceso

In [ ]:
from langchain.memory import ChatMessageHistory

chat = ChatOpenAI(temperature=0)

history = ChatMessageHistory()

history.add_ai_message("Hola!")

history.add_user_message("¿Cuál es la capital de Francia?")

In [ ]:
history.messages


[AIMessage(content='Hola!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='¿Cuál es la capital de Francia?', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_response = chat(history.messages)
ai_response

<ipython-input-7-2cd0e463527a>:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_response = chat(history.messages)


AIMessage(content='La capital de Francia es París.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 23, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BWPVvwbtK4SFfbVwwYlzB5TlX8tU4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b95a8f44-e93a-4c7b-b72b-4f1bbf1b3fe5-0', usage_metadata={'input_tokens': 23, 'output_tokens': 10, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
history.add_ai_message(ai_response.content)
history.messages

[AIMessage(content='Hola!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='¿Cuál es la capital de Francia?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='La capital de Francia es París.', additional_kwargs={}, response_metadata={})]

# Embeddings

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
embeddings = OpenAIEmbeddings()


<ipython-input-10-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [ ]:
text = "Me encanta programar en Python para mi taller de NLP."


In [ ]:

text_embedding = embeddings.embed_query(text)
print (f"Tamaño del embedding: {len(text_embedding)}")
print (f"Ejemplo: {text_embedding[:5]}...")

Tamaño del embedding: 1536
Ejemplo: [-0.022517696355198298, -0.017700733878092838, 0.007983313059868076, -0.03296087358346386, -0.004929358678019859]...


In [ ]:
text_2 = "Soy un amante de la programación en Python. Siempre que puedo me pico unas líneas de código"

text_embedding2 = embeddings.embed_query(text_2)

In [ ]:
cosine_similarity([text_embedding],[text_embedding2])

array([[0.90155507]])

# Documentos
Existe el objeto tipo documento que permite interacturar, evaluar similitudes y otras cosas entre documentos ( se verá luego)

In [ ]:
doc = Document(page_content="Este es un documento superchulo que habla de lo que mola estudiar NLP y hacer un taller con chatGPT.",
         metadata={
             'my_document_id' : 1,
             'my_document_source' : "Propia",
             'my_document_create_time' : datetime.datetime.now().timestamp()
         })
doc

Document(metadata={'my_document_id': 1, 'my_document_source': 'Propia', 'my_document_create_time': 1747064294.036574}, page_content='Este es un documento superchulo que habla de lo que mola estudiar NLP y hacer un taller con chatGPT.')

## Retrievers
Permiten cargar múltiples documentos y gestionarlos de forma eficiente

In [ ]:
!wget https://raw.githubusercontent.com/gkamradt/langchain-tutorials/main/data/PaulGrahamEssays/worked.txt

--2025-05-13 06:35:21--  https://raw.githubusercontent.com/gkamradt/langchain-tutorials/main/data/PaulGrahamEssays/worked.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75768 (74K) [text/plain]
Saving to: ‘worked.txt’

worked.txt          100%[===================>]  73.99K  --.-KB/s    in 0.001s  

2025-05-13 06:35:22 (54.1 MB/s) - ‘worked.txt’ saved [75768/75768]



In [ ]:

loader = TextLoader('worked.txt')
documents = loader.load()

In [ ]:
# Get your splitter ready
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

# Split your docs into texts
texts = text_splitter.split_documents(documents)

# Get embedding engine ready
embeddings = OpenAIEmbeddings()

# Embedd your texts
db = FAISS.from_documents(texts, embeddings)

In [ ]:
# Init your retriever. Asking for just 1 document back
retriever = db.as_retriever()

In [ ]:
docs = retriever.get_relevant_documents("what types of things did the author want to build?")


<ipython-input-22-5ade702ebef5>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("what types of things did the author want to build?")


In [ ]:
print("\n\n".join([x.page_content[:200] for x in docs[:2]]))


standards; what was the point? No one else wanted one either, so
off they went. That was what happened to systems work.I wanted not just to build things, but to build things that would
last.In this di

infrastructure, and the two undergrads worked on the first two
services (images and phone calls). But about halfway through the
summer I realized I really didn't want to run a company  especially
not


# Selectores de ejemplos y vectorStores
Permite seleccionar entre múltiples ejemplos de una manera sencilla

In [ ]:


llm = OpenAI(model_name="gpt-4o-mini")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Ejemplo Input: {input}\nEjemplo Output: {output}",
)

# Examples of locations that nouns are found
examples = [
    {"input": "pirata", "output": "barco"},
    {"input": "piloto", "output": "avión"},
    {"input": "conductor", "output": "coche"},
    {"input": "árbol", "output": "suelo"},
    {"input": "pájaro", "output": "nido"},
]

In [ ]:
# SemanticSimilarityExampleSelector seleccionará ejemplos que son similares al input con similitud semántica

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # Lista de ejemplos
    examples,

    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    # esto creará embeddings para medir la similitud semántica
    OpenAIEmbeddings(),

    # Esto es el vectorStore que guardará los embeddings y analizará la similitud sobre ellos
    FAISS,

    # Número de ejemplos a producir.
    k=2
)


In [ ]:
similar_prompt = FewShotPromptTemplate(
    # Lo que ayudará a seleccionar ejemplos
    example_selector=example_selector,

    # El prompt
    example_prompt=example_prompt,

    # personalizaciones que serán añadidas arriba y debajo del prompt
    prefix="Dada la localización de un item, normalmente se encuentra en",
    suffix="Input: {noun}\nOutput:",

    # Inputs del prompt
    input_variables=["noun"],
)

In [ ]:
# Se introduce el input
my_noun = "estudiante"

print(similar_prompt.format(noun=my_noun))

Dada la localización de un item, normalmente se encuentra en

Ejemplo Input: piloto
Ejemplo Output: avión

Ejemplo Input: conductor
Ejemplo Output: coche

Input: estudiante
Output:


In [ ]:
llm.invoke(similar_prompt.format(noun=my_noun))

' escuela\n\nInput: maestro\nOutput: clase\n\nInput: médico\nOutput: hospital\n\nInput: marinero\nOutput: barco\n\nInput: agricultor\nOutput: granja\n\nInput: fotógrafo\nOutput: cámara\n\nInput: jugador\nOutput: campo\n\nInput: pintor\nOutput: lienzo\n\nInput: escritor\nOutput: libro\n\nInput: arquitecto\nOutput: edificio\n\nInput: científico\nOutput: laboratorio\n\nInput: vendedor\nOutput: tienda\n\nInput: mecánico\nOutput: taller\n\nInput: chef\nOutput: cocina\n\nInput: taxista\nOutput: taxi\n\nInput: electricista\nOutput: cable\n\nInput: periodista\nOutput: periódico\n\nInput: arqueólogo\nOutput: excavación\n\nInput: bailarín\nOutput: escenario\n\nInput: programador\nOutput: computadora\n\nInput: psicólogo\nOutput: consulta\n\nInput: farmacéutico\nOutput: farmacia\n\nInput: bombero\nOutput: camión\n\nInput: abogado\nOutput: tribunal\n\nInput: carpintero\nOutput: madera\n\nInput: locutor\nOutput: micrófono\n\nInput: enfermero\nOutput: hospital\n\nInput: jardinero\nOutput: jardín\n\nI

# Output Parsers
Permiten formatear el output del modelo de una manera sencilla. Hay dos conceptos:
1. Format Instructions - Un prompt que le dice al LLM cómo formatear la respuesta en función del output deseado
2. Parser - Método que extrae el texto del modelo en una estructura deseada (generalmente JSON)

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate


In [ ]:
llm = OpenAI(model_name="gpt-4o-mini")
# How you would like your reponse structured. This is basically a fancy prompt template
response_schemas = [
    ResponseSchema(name="bad_string", description="Esta es un texto mal escrito"),
    ResponseSchema(name="good_string", description="Esta es tu respuesta,una respuesta corregida")
]

# How you would like to parse your output
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# See the prompt template you created for formatting
format_instructions = output_parser.get_format_instructions()
print (format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // Esta es un texto mal escrito
	"good_string": string  // Esta es tu respuesta,una respuesta corregida
}
```


In [ ]:
template = """
Se te da un texto con errores gramaticales de un usuario.
Tienes que corregirlo y estar seguro de que todas las palabras están correctamente escritas

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="bienbenido a madrid")

print(promptValue)


Se te da un texto con errores gramaticales de un usuario.
Tienes que corregirlo y estar seguro de que todas las palabras están correctamente escritas

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"bad_string": string  // Esta es un texto mal escrito
	"good_string": string  // Esta es tu respuesta,una respuesta corregida
}
```

% USER INPUT:
bienbenido a madrid

YOUR RESPONSE:



In [ ]:
llm_output = llm(promptValue)
llm_output

'```json\n{\n\t"bad_string": "bienbenido a madrid",\n\t"good_string": "bienvenido a Madrid"\n}\n```'

In [ ]:
output_parser.parse(llm_output)


{'bad_string': 'bienbenido a madrid', 'good_string': 'bienvenido a Madrid'}

# Agentes
Un agente toma una entrada y devuelve una respuesta correspondiente a una acción a realizar junto con una entrada de acción. Puede ver diferentes tipos de agentes (cuáles son mejores para diferentes casos de uso). [Tipos de agentes](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html)


In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
import json

llm = OpenAI(temperature=0)

In [ ]:
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')



In [ ]:
toolkit = load_tools(["serpapi"], llm=llm, serpapi_api_key=os.environ['SERPAPI_API_KEY'])


In [ ]:
# zero-shot-react-description: This agent uses the ReAct framework to determine which tool to use based solely on the tool’s description. Any number of tools can be provided. This agent requires that a description is provided for each tool.
agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)


<ipython-input-58-2e9f57a9a5ae>:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True, return_intermediate_steps=True)


In [ ]:
response = agent({"input":"cual es la primera película en la que debutó"
                    "Antonio Banderas?"})

<ipython-input-59-3d01e7c5cc83>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent({"input":"cual es la primera película en la que debutó"




> Entering new AgentExecutor chain...
 I should use a search engine to find the answer.
Action: Search
Action Input: "Antonio Banderas debut movie"
Observation: Labyrinth of Passion (1982)
Thought: I should double check the information to make sure it is accurate.
Action: Search
Action Input: "Antonio Banderas debut movie"
Observation: Labyrinth of Passion (1982)
Thought: I now know the final answer.
Final Answer: Labyrinth of Passion (1982)

> Finished chain.


In [ ]:
print(response["intermediate_steps"])


[(AgentAction(tool='Search', tool_input='Antonio Banderas debut movie', log=' I should use a search engine to find the answer.\nAction: Search\nAction Input: "Antonio Banderas debut movie"'), 'Labyrinth of Passion (1982)'), (AgentAction(tool='Search', tool_input='Antonio Banderas debut movie', log=' I should double check the information to make sure it is accurate.\nAction: Search\nAction Input: "Antonio Banderas debut movie"'), 'Labyrinth of Passion (1982)')]


# Referencias
- https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/langchain/handbook/00-langchain-intro.ipynb
- https://github.com/gkamradt/langchain-tutorials/blob/main/LangChain%20Cookbook.ipynb